In [9]:
x = ezdxf.readfile("../dxffiles/F15-TSR-TSM-Master-04.dxf")
for i in x.entities:
    if i.dxftype() == "LINE":
        break

In [12]:
i.dxf.start

Vec3(2131.60765277183, -675.4527248821383, 0.0)

In [15]:
ezdxf.math.Vec3.from_deg_angle(90)

Vec3(6.123233995736766e-17, 1.0, 0.0)

In [9]:
import os, ezdxf

# possibly merge all the 170color files into one (instead of by directory)
dfiles = "../dxffiles/aamaout/170color"
dfs = [ ]
for f in os.listdir(dfiles):
    if f[0] != "#" and f[0] != "p" and os.path.splitext(f)[1] == ".dxf":
        df = os.path.join(dfiles, f)
        dfs.append(df)

In [12]:
from dxfgrouping import linearizeelement
import json

js = [ ]
for df in dfs:
    d = ezdxf.readfile(df)
    for b in d.entities:
        assert b.dxftype() == "INSERT"
        j = { "blockname":b.dxf.name, "blockrotation":b.dxf.rotation, "blockposition":list(b.dxf.insert)[:2] }
        bb = d.blocks[b.dxf.name]
        outercut = bb.entity_space.entities[0]
        assert outercut.dxftype() == "POLYLINE"
        assert outercut.dxf.layer == "1"
        outercontour = [ list(v.dxf.location)[:2]  for v in outercut.vertices ]
        assert outercontour[0] == outercontour[-1]
        j["outercontour"] = outercontour
        innerlines = [ ]
        for e in bb.entity_space.entities[1:]:
            innerlines.append([ list(p)[:2]  for p in linearizeelement(e) ])
        j["innerlines"] = innerlines
        js.append(j)
        

In [14]:
json.dump(js, open(os.path.join(dfiles, "shapes.json"), "w"))

In [35]:
jps = json.load(open(os.path.join(dfiles, "positions.json"), "r"))
outputfilename = os.path.join(dfiles, "positions.dxf")
doc = ezdxf.new('R12')
msp = doc.modelspace()
aamacutlayer = doc.layers.new("1", {"color":1})
aamadrawlayer = doc.layers.new("8", {"color":4})
aamaintcutlayer = doc.layers.new("11", {"color":3})
for df in dfs:
    d = ezdxf.readfile(df)
    for b in d.entities:
        assert b.dxftype() == "INSERT"
        bb = d.blocks[b.dxf.name]
        block = doc.blocks.new(name=b.dxf.name)
        for e in bb.entity_space.entities:
            block.add_foreign_entity(e)
        bp = jps[b.dxf.name]
        dxfattribs = {'rotation':bp[2], 'linetype':'BYLAYER' }
        msp.add_blockref(b.dxf.name, ezdxf.math.vector.Vector(bp[0], bp[1], 0), dxfattribs=dxfattribs)
        
    doc.set_modelspace_vport(height=2300, center=(1800, 900))
    doc.saveas(outputfilename)
